In [5]:
#import necessary libraries and files
import pandas as pd
from faker import Faker
import numpy as np
import random
from datetime import datetime, timedelta
import os
import time

from utils.create_rules import run_rules
from utils.awsPostgresConnection import get_db_connection
from configurations.config import merchant_categories, card_types, num_records

In [8]:
#connect to database and Create the banking_data table if it does not exist
conn = get_db_connection()
cur = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS banking_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMPTZ NOT NULL,
    uniq_id UUID NOT NULL,
    trans_type VARCHAR(50) NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    amount_crr DECIMAL(10, 2) NOT NULL,
    account_holder_name VARCHAR(100) NOT NULL,
    card_presense VARCHAR(50) NOT NULL,
    merchant_category VARCHAR(50) NOT NULL,
    card_type VARCHAR(50) NOT NULL,
    card_id VARCHAR(20) NOT NULL,
    account_id UUID NOT NULL,
    account_blacklisted BOOLEAN NOT NULL,
    rules_triggered VARCHAR(100),
    rules_explanation VARCHAR(100),
    decision VARCHAR(100) 
);
"""
cur.execute(create_table_query)
conn.commit()

In [9]:
#initialize faker library
fake =Faker()
#function to generate a single data
def generate_record():
    card_type = random.choice(list(card_types.keys()))
    return {
        "uniq_id": [fake.uuid4()],
        "trans_type": [random.choice(["Real_time_transaction", "settlements", "dispute"])],
        "amount": [round(random.uniform(10.0, 1000.0), 2)],
        "amount_crr": [round(random.uniform(10.0, 1000.0), 2)],
        "account_holder_name": [fake.name()],
        "card_presense": [random.choice(["Present", "Not Present"])],
        "merchant_category": [random.choice(merchant_categories)],
        "card_type": [card_type],
        "card_id": [fake.credit_card_number(card_type=card_types[card_type])],
        "account_id": [fake.uuid4()],
        "account_blacklisted": [random.choice([True, False])]
    }

In [10]:
while True:
    timestamp = datetime.utcnow()
    
    for _ in range(num_records):
        temp = generate_record()
        df = pd.DataFrame(temp)
        record = run_rules(df)
        cur.execute("""
        INSERT INTO banking_data (timestamp, uniq_id, trans_type, amount, amount_crr, account_holder_name, card_presense, merchant_category, card_type, card_id, account_id, account_blacklisted,rules_triggered,rules_explanation,decision) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s)""",
                    (timestamp, record['uniq_id'], record['trans_type'], record['amount'], record['amount_crr'],
                     record['account_holder_name'], record['card_presense'], record['merchant_category'],
                     record['card_type'], record['card_id'], record['account_id'], record['account_blacklisted'],record['rules_triggered'],record['rules_explanation'],record['decision']))
        conn.commit()

    time.sleep(15)  # Adjust the sleep duration as needed, here it's set to 15 seconds

cur.close()
conn.close()

C:\Users\Vaishnavi\AppData\Local\Temp\ipykernel_36244\3378784078.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow()
C:\Users\Vaishnavi\AppData\Local\Temp\ipykernel_36244\3378784078.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow()
C:\Users\Vaishnavi\AppData\Local\Temp\ipykernel_36244\3378784078.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow()
C:\Users\Vaishnavi\AppData\Local\Temp\ipykernel_36244\3378784078.py:2: Deprecation

KeyboardInterrupt: 